In [6]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx

In [30]:
fig_idx = 0
def new_fig():
    global fig_idx
    plt.figure(fig_idx)
    fig_idx += 1

def sort_degree_centrality_desc(G):
    degree_cetrality = dict(nx.degree_centrality(G))
    return {k: v for k, v in sorted(degree_cetrality.items(), key=lambda item: item[1], reverse=True)}

def draw_interdependent_G(G):
    global ATTR_LAYER, LAYER_A_TAG, LAYER_B_TAG
    layer_attr=nx.get_node_attributes(G,ATTR_LAYER)
    color_map = ['b' if layer_attr[node][0]== LAYER_A_TAG else 'g' for node in list(G.nodes())]
    new_fig()
    nx.draw(G, node_color=color_map, node_size=30)
### Singe Random Network
### network config
AVG_DEGREE = 4
NUM_NODE = 10
NODE_DESTROY_THRESHOLD = 0.2
ATTACK_NODE_NUM = 1
M_MONTE_CARLO = 100

ATTR_LAYER = "layer"
LAYER_A_TAG = "L_A"
LAYER_B_TAG = "L_B"

ATTR_STATE = "state"
UNHEALTHY_STATE = "unhealthy"
###
### with certain average degree
##### ER
def generate_ER(avg_degree):
    p = np.true_divide(avg_degree, NUM_NODE)
    return nx.erdos_renyi_graph(NUM_NODE, p)
#### WS
def generate_WS(avg_degree):
    pass
#### BA
def generate_BA(avg_degree):
    pass

### inter-connect
### add properties of whether nodes is Layer A or B for further use
### return: G
####
#### e.g. max_max means nodes in layer A with highest degree centralities
####         connect to  nodes in layer A with highest degree centralities
def interconnect_random(G_layer_A, G_layer_B):
    pass
def interconnect_by_degree_centralities(G_layer_A, L_A_degree_c, G_layer_B, L_B_degree_c ):
    G = nx.union(G_layer_A, G_layer_B, rename=(LAYER_A_TAG+"_", LAYER_B_TAG+"_"))
    for idx in range(len(L_A_degree_c.items())):
        node_A = LAYER_A_TAG+"_"+str(list(L_A_degree_c.keys())[idx])
        node_B = LAYER_B_TAG+"_"+str(list(L_B_degree_c.keys())[idx])
        G.add_edge(node_A, node_B)
    return G
def interconnect_max_max(G_layer_A, G_layer_B):
    global LAYER_A_TAG, LAYER_B_TAG
    L_A_degree_c = sort_degree_centrality_desc(G_layer_A)
    L_B_degree_c = sort_degree_centrality_desc(G_layer_B)
    return interconnect_by_degree_centralities(G_layer_A, L_A_degree_c, G_layer_B, L_B_degree_c)
def interconnect_max_min(G_layer_A, G_layer_B):
    pass
def interconnect_min_max(G_layer_A, G_layer_B):
    pass
def interconnect_min_min(G_layer_A, G_layer_B):
    pass
###### intersimilarity refers to https://doi.org/10.1103/PhysRevE.88.052805
###### Layer A node X's neighbours tend to connect to the neighbours of the nodes in Layer B which connects to X
def interconnect_inter_similarity(G_layer_A, G_layer_B):
    pass
#### simulation
def simulate(interdependent_G:nx.Graph, attack_nodes:[], measure_fuction):
    global M_MONTE_CARLO, NODE_DESTROY_THRESHOLD, UNHEALTHY_STATE, ATTR_STATE
    G = interdependent_G.copy()
    current_atk = []
    for node in attack_nodes:
        G.nodes()[node][ATTR_STATE] = UNHEALTHY_STATE
        n = G.neighbors(node)
        for nb in n:
            current_atk.append(nb)
    MMC_pointer = 0
    while MMC_pointer<M_MONTE_CARLO:
        next = []
        # get neighbours
        for node in current_atk:
            if (G.nodes()[node].get(ATTR_STATE, "") == UNHEALTHY_STATE) :
                continue
            neighbours = G.neighbors(node)
            neighbours_list = list(neighbours)
            if len(neighbours_list)==0:
                continue
            unhealthy_count = 0
            for n in neighbours_list:
                if (G.nodes()[n].get(ATTR_STATE, "") == UNHEALTHY_STATE):
                    unhealthy_count+=1
            if (np.true_divide(unhealthy_count, len(neighbours_list)) >= NODE_DESTROY_THRESHOLD):
                G.nodes()[node][ATTR_STATE] = UNHEALTHY_STATE
                for neighbour in neighbours_list:
                    next.append(neighbour)
        current_atk = next
        MMC_pointer+=1
    pass
#### measurement
def measurement(current_G):
    pass
#### attack nodes
def attack_random_Layer_A(G_interdependent):
    global ATTACK_NODE_NUM

    return []
def attack_random_Layer_A_B(G_interdependent):
    global ATTACK_NODE_NUM

    return []
def attack_high_degree_centrality_Layer_A(G_interdependent):
    global  LAYER_A_TAG
    degree_c = sort_degree_centrality_desc(G_interdependent)
    for i in list(degree_c.items()):
        if (LAYER_A_TAG in i[0]):
            return  [i[0]]
    return [LAYER_A_TAG+"_0"]
def attack_high_degree_centrality_Layer_A_B(G_interdependent):
    return []

def set_Layer_tag(G:nx.Graph, tag):
    labels = [tag]
    global ATTR_LAYER
    nx.set_node_attributes(G, labels, ATTR_LAYER)
    return G
##### experiment: Failure cascade
def experiment(layer_A_function, layer_B_function, interconnect_function, attack_strategy):
    global AVG_DEGREE
    # Build interdependent networks
    ### construct Layer A
    G_A = layer_A_function(AVG_DEGREE)
    set_Layer_tag(G_A, LAYER_A_TAG)
    ### construct Layer B
    G_B = layer_B_function(AVG_DEGREE)
    set_Layer_tag(G_B, LAYER_B_TAG)

    ### interconnect
    G = interconnect_function(G_A, G_B)

    # select nodes to attack
    attack_nodes = attack_strategy(G)

    # simulate cascade
    simulate(G, attack_nodes, lambda curr_G:measurement(curr_G))
    pass


experiment(lambda avg_d:generate_ER(avg_d),lambda avg_d:generate_ER(avg_d),
           lambda G_A,G_B: interconnect_max_max(G_A, G_B),
           lambda G_inter: attack_high_degree_centrality_Layer_A(G_inter))